# Tópico de Industria I - Assignment 3 - Dashboard
Izmael Franco Quezada

Minimum requirements:
- Functions that return the different types of trajectories: Brownian Motion (BM),Correlated Random Walk (CRW), and Lévy Flight (LF).
- All functions must take as arguments the number of steps, speed, and starting position.
 - In addition, CRW and LF functions must also take as an argument the Cauchy coefficient.
 - Lastly, the LF function must also include the Lévy exponent (alpha) as an argument.

- Functions that compute the metrics: Path length (PL), Mean Squared Displacement (MSD), and Turning Angle Distribution (TAD).

# Librerias

In [28]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from scipy.stats import levy_stable, cauchy
import math
import ipywidgets as widgets
from IPython.display import display

# Vec2d Class

In [29]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Main code

In [31]:
def brownian_motion_trajectory(n_steps, speed, initial_pos):
    """Simulates a 2D trajectory using Brownian motion."""
    trajectory = np.zeros((n_steps, 2))
    position = Vec2d(initial_pos[0], initial_pos[1])
    
    for i in range(n_steps):
        # Generate small random steps in x and y
        delta_x = np.random.normal(loc=0, scale=speed)
        delta_y = np.random.normal(loc=0, scale=speed)
        step_vector = Vec2d(delta_x, delta_y)
        
        position += step_vector
        trajectory[i] = [position.x, position.y]
    
    return pd.DataFrame(trajectory, columns=["x", "y"])

############## 3D Visualization Function for Brownian Motion ################
def visualize_brownian_motion_3d(n_steps, speed, initial_x, initial_y):
    # Generate the trajectory
    initial_pos = [initial_x, initial_y]
    trajectory_df = brownian_motion_trajectory(n_steps, speed, initial_pos)
    
    # Add 'z' as the number of steps
    trajectory_df['z'] = np.arange(len(trajectory_df))
    
    # Visualize the trajectory in 3D using Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory_df['x'],
        y=trajectory_df['y'],
        z=trajectory_df['z'],
        mode='lines',
        line=dict(color='green', width=2)
    )])
    
    fig.update_layout(
        title='3D Trajectory',
        scene=dict(
            xaxis_title='X_pos',
            yaxis_title='Y_pos',
            zaxis_title='Step Number'
        )
    )
    
    fig.show(renderer="notebook") 

############## Lévy Flight Trajectory Function ################
def levy_flight_trajectory(n_steps, alpha, beta, initial_pos, speed):
    """Simulate a 2D Lévy flight trajectory."""
    position = Vec2d(initial_pos[0], initial_pos[1])
    trajectory = [(position.x, position.y)]  # Initial position in trajectory
    
    for _ in range(n_steps):
        step_length = levy_stable.rvs(alpha, beta) * speed
        angle = np.random.uniform(0, 2 * np.pi)
        step_vector = Vec2d(step_length * np.cos(angle), step_length * np.sin(angle))
        position += step_vector
        trajectory.append((position.x, position.y))
    
    return pd.DataFrame(trajectory, columns=["x", "y"])

############## 3D Visualization Function for Lévy Flight ################
def visualize_trajectory_3d(n_steps, speed, initial_x, initial_y, cauchy_coef):
    # Generate the trajectory
    alpha, beta = 1.5, cauchy_coef  # Use the Cauchy coefficient in the Lévy-Stable distribution
    initial_pos = [initial_x, initial_y]
    trajectory_df = levy_flight_trajectory(n_steps, alpha, beta, initial_pos, speed)
    
    # Add 'z' as the number of steps
    trajectory_df['z'] = np.arange(len(trajectory_df))
    
    # Visualize with Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory_df['x'],
        y=trajectory_df['y'],
        z=trajectory_df['z'],
        mode='lines',
        line=dict(color='blue', width=2)
    )])
    
    fig.update_layout(
        title='3D Trajectory',
        scene=dict(
            xaxis_title='X_pos',
            yaxis_title='Y_pos',
            zaxis_title='Step Number'
        )
    )
    
    fig.show(renderer="notebook") 

############## 3D Visualization Function for CRW (Cauchy Distribution) ################
def visualize_trajectory_CRW_3d(n_steps, initial_x, initial_y):
    # Initialize the trajectory
    trajectory = np.zeros((n_steps, 2))  # Trajectory in a numpy array
    initial_position = Vec2d(initial_x, initial_y)
    initial_vector = Vec2d(1, 0)

    # Random angles using the Cauchy distribution
    cauchy_rvs = cauchy.rvs(size=n_steps)  # Use cauchy.rvs directly
    current_position = initial_position
    current_vector = initial_vector

    for i in range(n_steps):
        angle = cauchy_rvs[i]  # Take the random angle
        current_vector = current_vector.rotated(angle)
        current_position = current_position + current_vector
        trajectory[i] = [current_position.x, current_position.y]  # Store the new position

    # Add 'z' as the number of steps
    z = np.arange(n_steps)

    # Visualize the trajectory in 3D using Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=trajectory[:, 0],
        y=trajectory[:, 1],
        z=z,
        mode='lines',  # Only show lines
        line=dict(color='red', width=2)  # Line settings
    )])

    fig.update_layout(
        title='3D Trajectory',
        scene=dict(
            xaxis_title='X_position',
            yaxis_title='Y_position',
            zaxis_title='Step Number'
        )
    )

    fig.show(renderer="notebook")

############## Create an Interactive Panel ################
# Interactive widgets
n_steps_slider = widgets.IntSlider(value=1000, min=100, max=5000, step=100, description='Steps')
speed_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Speed')
initial_x_text = widgets.FloatText(value=0.0, description='Initial X')
initial_y_text = widgets.FloatText(value=0.0, description='Initial Y')
cauchy_coef_slider = widgets.FloatSlider(value=0.0, min=0, max=2.0, step=0.1, description='Cauchy Coef')

# Button to select the simulation
levy_button = widgets.RadioButtons(
    options=['Levy 3D Trajectory', 'CRW 3D Trajectory', 'Brownian Motion 3D Trajectory'],
    description='Select Simulation:',
    disabled=False
)

# Function to dynamically update the graph
def update_graph(n_steps, speed, initial_x, initial_y, cauchy_coef, simulation):
    if simulation == 'Levy 3D Trajectory':
        visualize_trajectory_3d(n_steps, speed, initial_x, initial_y, cauchy_coef)
    elif simulation == 'CRW 3D Trajectory':
        visualize_trajectory_CRW_3d(n_steps, initial_x, initial_y)
    elif simulation == 'Brownian Motion 3D Trajectory':
        visualize_brownian_motion_3d(n_steps, speed, initial_x, initial_y)

# Interactive panel that updates based on parameters
interactive_output = widgets.interactive_output(
    update_graph,
    {
        'n_steps': n_steps_slider,
        'speed': speed_slider,
        'initial_x': initial_x_text,
        'initial_y': initial_y_text,
        'cauchy_coef': cauchy_coef_slider,
        'simulation': levy_button
    }
)

# Display all controls
display(
    levy_button,
    n_steps_slider,
    speed_slider,
    initial_x_text,
    initial_y_text,
    cauchy_coef_slider,
    interactive_output
)


RadioButtons(description='Select Simulation:', options=('Levy 3D Trajectory', 'CRW 3D Trajectory', 'Brownian M…

IntSlider(value=1000, description='Steps', max=5000, min=100, step=100)

FloatSlider(value=1.0, description='Speed', max=5.0, min=0.1)

FloatText(value=0.0, description='Initial X')

FloatText(value=0.0, description='Initial Y')

FloatSlider(value=0.0, description='Cauchy Coef', max=2.0)

Output()